# Evaluation of VLM Responses (Only for GPT Judge) ~ For Our Manuscript we had Agriculture Experts Analyze the Results

**Metrics**
- **Weed Detection Accuracy**: Strict Yes/No match between `Weed Detection (Prediction)` and `Weed Detection (GT)`.
- **Crop Growth Accuracy (Adjacent)**: treats growth as an ordered scale and counts predictions within **1 step** as correct.
- **Crop Type Fuzzy-Accuracy @ 80%**: similarity between `Crop Type (Prediction)` and `Crop Type (GT)` using difflib; counts as correct if similarity ≥ 0.80 (after normalization).


In [ ]:
# Inline-only evaluation (no files written)
import os, re
import numpy as np
import pandas as pd
from difflib import SequenceMatcher

# ================== CONFIG ==================
excel_path = "/Complete_Weed_VLM_Evaluation(REFINED).xlsx"

# Column names
COL_WEED_GT   = "Weed Detection (GT)"
COL_WEED_PRED = "Weed Detection (Prediction)"
COL_GROWTH_PRED = "Crop Growth (Prediction)"
COL_RESPONSE  = "GPT_Response"
COL_CROP_GT   = "Crop Type (GT)"
COL_CROP_PRED = "Crop Type (Prediction)"

# Growth normalization
GROWTH_ORDER = ["seedling", "young", "growing", "mature", "full grown"]
GROWTH_SYNONYMS = {
    "seed": "seedling", "sprout": "seedling", "seedling": "seedling",
    "young": "young", "early": "young",
    "growing": "growing", "growth": "growing", "mid": "growing",
    "mature": "mature", "ripened": "mature",
    "full grown": "full grown", "full-grown": "full grown",
    "fully grown": "full grown", "harvest-ready": "full grown", "harvest ready": "full grown",
}

def to_str(x):
    if pd.isna(x): return ""
    return str(x).strip()

def normalize_yes_no(x):
    s = re.sub(r'[^a-z]', '', to_str(x).lower())
    if s in {"yes","y","true"}: return 1
    if s in {"no","n","false"}: return 0
    return None

def normalize_growth(x):
    s = to_str(x).lower()
    s = re.sub(r'\s+', ' ', s).replace("_"," ").replace("-"," ").strip()
    if not s: return None
    if s in GROWTH_ORDER: return s
    toks = s.split()
    if "full" in toks and "grown" in toks: return "full grown"
    for t in toks:
        if t in GROWTH_SYNONYMS: return GROWTH_SYNONYMS[t]
    # fuzzy fallback to nearest label
    best,score=-1,None
    for i,g in enumerate(GROWTH_ORDER):
        r = SequenceMatcher(None, s, g).ratio()
        if score is None or r>score:
            score=r; best=g
    return best

def normalize_crop_type(x):
    s = to_str(x).lower()
    s = re.sub(r'[^a-z0-9\s/+-]', '', s)
    s = re.sub(r'\s+',' ', s).strip()
    if not s: return ""
    if s.endswith('ies'): s = s[:-3] + 'y'
    elif s.endswith('ses'): s = s[:-2]
    elif s.endswith('s') and not s.endswith('ss'): s = s[:-1]
    aliases = {"soybean":"soybean","soy":"soybean","soyabean":"soybean","maize":"corn",
               "corn":"corn","wheat":"wheat","barley":"barley","rice":"rice","unknown":"unknown"}
    return aliases.get(s, s)

def extract_from_response(text, field):
    """
    Extract value like 'Crop Growth: Growing' from Gemini_Response free text.
    field examples: 'Crop Growth', 'Crop Type'.
    """
    s = to_str(text)
    # capture until line break or sentence end
    pat = rf'(?is){re.escape(field)}\s*:\s*([A-Za-z \-_/]+?)($|\n|\r|\.|,)'  # non-greedy
    m = re.search(pat, s)
    if m: return m.group(1).strip()
    return ""

def safe_acc(matches):
    if not matches: return float('nan')
    return float(sum(matches) / len(matches))

def evaluate_sheet(df, sheet):
    # 1) Weed Detection: Prediction vs GT
    wd_matches = []
    if COL_WEED_GT in df.columns and COL_WEED_PRED in df.columns:
        for _, row in df[[COL_WEED_GT, COL_WEED_PRED]].iterrows():
            g = normalize_yes_no(row[COL_WEED_GT])
            p = normalize_yes_no(row[COL_WEED_PRED])
            if g is None or p is None: continue
            wd_matches.append(g==p)
    weed_acc = safe_acc(wd_matches); n_wd=len(wd_matches)

    # 2) Crop Growth: Prediction vs Response (parsed)
    cg_matches = []
    if COL_GROWTH_PRED in df.columns and COL_RESPONSE in df.columns:
        for _, row in df[[COL_GROWTH_PRED, COL_RESPONSE]].iterrows():
            pred = normalize_growth(row[COL_GROWTH_PRED])
            resp_raw = extract_from_response(row[COL_RESPONSE], "Crop Growth")
            resp = normalize_growth(resp_raw)
            if pred is None or resp is None: continue
            cg_matches.append(pred==resp)
    cg_acc = safe_acc(cg_matches); n_cg=len(cg_matches)

    # 3) Crop Type: Prediction vs GT (strict, normalized)
    ct_matches = []
    if COL_CROP_GT in df.columns and COL_CROP_PRED in df.columns:
        for _, row in df[[COL_CROP_GT, COL_CROP_PRED]].iterrows():
            g = normalize_crop_type(row[COL_CROP_GT])
            p = normalize_crop_type(row[COL_CROP_PRED])
            if not g or not p: continue
            ct_matches.append(g==p)
    ct_acc = safe_acc(ct_matches); n_ct=len(ct_matches)

    return {
        "sheet": sheet,
        "weed_detection_accuracy": weed_acc, "weed_n": n_wd,
        "crop_growth_accuracy": cg_acc, "growth_n": n_cg,
        "crop_type_accuracy": ct_acc, "crop_type_n": n_ct,
    }

def evaluate_workbook(path):
    xl = pd.ExcelFile(path)
    res = []
    for name in xl.sheet_names:
        df = xl.parse(name)
        res.append(evaluate_sheet(df, name))
    summary = pd.DataFrame(res)

    # weighted overall by valid counts
    def wavg(col_acc, col_n):
        w = summary[col_n].fillna(0).astype(float)
        a = summary[col_acc].fillna(0).astype(float)
        d = w.sum()
        if d==0: return float('nan')
        return float((w*a).sum()/d)

    overall = {
        "sheet": "__OVERALL__",
        "weed_detection_accuracy": wavg("weed_detection_accuracy","weed_n"),
        "weed_n": int(summary["weed_n"].sum() if "weed_n" in summary else 0),
        "crop_growth_accuracy": wavg("crop_growth_accuracy","growth_n"),
        "growth_n": int(summary["growth_n"].sum() if "growth_n" in summary else 0),
        "crop_type_accuracy": wavg("crop_type_accuracy","crop_type_n"),
        "crop_type_n": int(summary["crop_type_n"].sum() if "crop_type_n" in summary else 0),
    }
    full = pd.concat([summary, pd.DataFrame([overall])], ignore_index=True)
    return full

if not os.path.exists(excel_path):
    print("⚠️ Excel file not found at:", excel_path)
    print("Upload your workbook to /mnt/data and set excel_path accordingly.")
else:
    df_out = evaluate_workbook(excel_path)
    try:
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user("Inline Evaluation Summary", df_out)
    except Exception:
        pass
    print(df_out.to_string(index=False))


                 sheet  weed_detection_accuracy  weed_n  crop_growth_accuracy  growth_n  crop_type_accuracy  crop_type_n
      Gemini_Flash_2.5                 0.772834     427                   NaN         0            0.884793          434
 Gemini_Flash_Lite_2.5                 0.682540     441                   NaN         0            0.895692          441
      LLaMA_4_Maverick                 0.455782     441                   NaN         0            0.482993          441
         LLaMA_4_Scout                 0.149660     441                   NaN         0            0.000000          441
           ChatGPT_4.1                 0.308390     441                   NaN         0            0.589569          441
            ChatGPT_4o                 0.455172     435                   NaN         0            0.671264          435
          Weed_Dataset                      NaN       0                   NaN         0            0.000000            1
LLaMA_4_Maverick (Old)          

In [ ]:
# ---- Configuration & Sheet Listing ----
import pandas as pd, os

# Set your Excel path here

excel_path = r"/Comprehensive_WeedVLM.xlsx"

# if not os.path.exists(excel_path):
#     print("⚠️ Excel file not found at:", excel_path)
# else:
#     xl = pd.ExcelFile(excel_path)
#     print("Found sheets:")
#     for i, name in enumerate(xl.sheet_names, start=1):
#         print(f"{i}. {name}")

# ---- Single-Sheet Evaluation ----
import os, re
import numpy as np
import pandas as pd

# Columns
COL_WEED_GT     = "Weed Detection (GT)"
COL_WEED_PRED   = "Weed Detection (Prediction)"
COL_GROWTH_GT   = "Crop Growth (GT)"
COL_GROWTH_PRED = "Crop Growth (Prediction)"
COL_CROP_GT     = "Crop Type (GT)"
COL_CROP_PRED   = "Crop Type (Prediction)"

# Choose ONE of these (name has priority if both set)
SHEET_NAME  = "Gemini_Flash_Lite_2.5"  # e.g., "Gemini_Flash_Lite_2.5", "LLaMA_4_Maverick", "LLaMA_4_Scout", "ChatGPT_4.1", "ChatGPT_4o"
SHEET_INDEX = None          # 1-based index, e.g., 2 for the second sheet

# ----- Normalizers -----
def to_str(x):
    if pd.isna(x): return ""
    return str(x).strip()

def norm_weed_label(x):
    """Return one of: 'yes', 'no', 'n/a', '' (empty)."""
    s_raw = to_str(x).lower()
    if s_raw == "":
        return ""
    letters = re.sub(r'[^a-z]', '', s_raw)  # drop spaces, punctuation
    if letters == "yes": return "yes"
    if letters == "no":  return "no"
    if letters == "na":  return "n/a"
    return s_raw.strip()

def norm_growth(x):
    """Map to one of: 'early', 'growing', 'full grown'."""
    s = to_str(x).lower()
    s = re.sub(r'[\s_-]+', ' ', s).strip()
    if not s: return None
    if s in {"early"}: return "early"
    if s in {"growing"}: return "growing"
    if s in {"full grown", "full-grown", "fully grown"}: return "full grown"
    if "full" in s and "grown" in s: return "full grown"
    return s

def norm_crop_type(x):
    """Strict, minimal normalization: case-insensitive + trim only."""
    return to_str(x).strip().lower()

def safe_acc(matches):
    if not matches: return float('nan')
    return float(sum(matches) / len(matches))

# ----- Load sheet -----
if not os.path.exists(excel_path):
    print("⚠️ Excel file not found at:", excel_path)
else:
    xl = pd.ExcelFile(excel_path)
    sheet_to_use = None
    if SHEET_NAME is not None and SHEET_NAME in xl.sheet_names:
        sheet_to_use = SHEET_NAME
    elif SHEET_INDEX is not None:
        try:
            idx0 = int(SHEET_INDEX) - 1
            if 0 <= idx0 < len(xl.sheet_names):
                sheet_to_use = xl.sheet_names[idx0]
        except Exception:
            sheet_to_use = None
    else:
        # default to first sheet if neither provided
        if len(xl.sheet_names) > 0:
            sheet_to_use = xl.sheet_names[0]

    if sheet_to_use is None:
        print("⚠️ Please set SHEET_NAME (exact) or SHEET_INDEX (1-based) to a valid sheet.")
    else:
        df = xl.parse(sheet_to_use)
        print(f"Evaluating sheet: {sheet_to_use} (rows: {len(df)})")

        # 1) Weed Detection accuracy
        wd_matches = []
        if COL_WEED_GT in df.columns and COL_WEED_PRED in df.columns:
            for _, row in df[[COL_WEED_GT, COL_WEED_PRED]].iterrows():
                gt = norm_weed_label(row[COL_WEED_GT])
                pr = norm_weed_label(row[COL_WEED_PRED])
                if gt in {"yes","no","n/a",""} and pr in {"yes","no","n/a",""}:
                    wd_matches.append(gt == pr)
        wd_acc = safe_acc(wd_matches)
        print(f"Weed Detection Accuracy: {wd_acc:.4f} (n={len(wd_matches)})")

        # 2) Crop Growth accuracy (Prediction vs GT; classes: Early, Growing, Full Grown)
        cg_matches = []
        if COL_GROWTH_GT in df.columns and COL_GROWTH_PRED in df.columns:
            for _, row in df[[COL_GROWTH_GT, COL_GROWTH_PRED]].iterrows():
                gt = norm_growth(row[COL_GROWTH_GT])
                pr = norm_growth(row[COL_GROWTH_PRED])
                if gt in {"early","growing","full grown"} and pr in {"early","growing","full grown"}:
                    cg_matches.append(gt == pr)
        cg_acc = safe_acc(cg_matches)
        print(f"Crop Growth Accuracy (Early/Growing/Full Grown): {cg_acc:.4f} (n={len(cg_matches)})")

        # 3) Crop Type accuracy (strict; no synonym/plural logic)
        ct_matches = []
        if COL_CROP_GT in df.columns and COL_CROP_PRED in df.columns:
            for _, row in df[[COL_CROP_GT, COL_CROP_PRED]].iterrows():
                gt = norm_crop_type(row[COL_CROP_GT])
                pr = norm_crop_type(row[COL_CROP_PRED])
                if gt != "" and pr != "":
                    ct_matches.append(gt == pr)
        ct_acc = safe_acc(ct_matches)
        print(f"Crop Type Accuracy (strict): {ct_acc:.4f} (n={len(ct_matches)})")


Evaluating sheet: Gemini_Flash_Lite_2.5 (rows: 450)
Weed Detection Accuracy: 0.6847 (n=444)
Crop Growth Accuracy (Early/Growing/Full Grown): 0.0000 (n=440)
Crop Type Accuracy (strict): 0.9048 (n=441)


## Refined Evaluation Code

In [ ]:
# ---- Single-Sheet Evaluation with Row Limits & EPP Metrics ----
import os, re
import numpy as np
import pandas as pd

# Set Excel path
excel_path = r"/Complete_Weed_VLM_Evaluation(REFINED).xlsx"

# Columns (by header)
COL_WEED_GT     = "Weed Detection (GT)"
COL_WEED_PRED   = "Weed Detection (Prediction)"
COL_GROWTH_GT   = "Crop Growth (GT)"
COL_GROWTH_PRED = "Crop Growth (Prediction)"
COL_CROP_GT     = "Crop Type (GT)"
COL_CROP_PRED   = "Crop Type (Prediction)"

# Choose ONE of these (name has priority if both set)
SHEET_NAME  = "ChatGPT_4.1"  # e.g., "Gemini_Flash_2.5", "Gemini_Flash_Lite_2.5", "LLaMA_4_Maverick", "LLaMA_4_Scout", "ChatGPT_4.1", "ChatGPT_4o"
SHEET_INDEX = None             # 1-based index (default: first sheet)

# Optional row limits (inclusive, 1-based)
DATA_ROW_START = 1
DATA_ROW_END   = 442

# ----- Normalizers -----
def to_str(x):
    if pd.isna(x): return ""
    return str(x).strip()

def norm_weed_label(x):
    """Return one of: 'yes', 'no', 'n/a', '' (empty)."""
    s_raw = to_str(x).lower()
    if s_raw == "": return ""
    letters = re.sub(r'[^a-z]', '', s_raw)
    if letters == "yes": return "yes"
    if letters == "no":  return "no"
    if letters == "na":  return "n/a"
    return s_raw.strip()

def norm_growth(x):
    """Map to one of: 'early', 'growing', 'full grown'."""
    s = to_str(x).lower()
    s = re.sub(r'[\s_-]+', ' ', s).strip()
    if not s: return None
    if s == "early": return "early"
    if s == "growing": return "growing"
    if s in {"full grown","full-grown","fully grown"}: return "full grown"
    if "full" in s and "grown" in s: return "full grown"
    return s

def norm_crop_type(x):
    """Strict compare: case-insensitive + trim only."""
    return to_str(x).lower()

def safe_acc(matches):
    if not matches: return float('nan')
    return float(sum(matches) / len(matches))

def excel_col_to_index(col_letters: str) -> int:
    """Convert Excel col letters (e.g., 'A') to 0-based index."""
    col = col_letters.strip().upper()
    total = 0
    for ch in col:
        total = total * 26 + (ord(ch) - ord('A') + 1)
    return total - 1

def series_by_col_letter(df: pd.DataFrame, col_letters: str):
    idx = excel_col_to_index(col_letters)
    if idx < 0 or idx >= df.shape[1]:
        return None, None
    return df.iloc[:, idx], df.columns[idx]

def apply_row_limits(df: pd.DataFrame):
    if DATA_ROW_START is None and DATA_ROW_END is None:
        return df.copy()
    start0 = max(0, DATA_ROW_START-1)
    end0   = min(len(df), DATA_ROW_END)
    return df.iloc[start0:end0].copy()

def accuracy_from_series(gt_ser, pr_ser, normalizer, allowed=None):
    matches = []
    for gt, pr in zip(gt_ser, pr_ser):
        g, p = normalizer(gt), normalizer(pr)
        if allowed is not None:
            if g not in allowed or p not in allowed: continue
        else:
            if g=="" or p=="": continue
        matches.append(g==p)
    return safe_acc(matches), len(matches)

# ---- Run evaluation ----
if not os.path.exists(excel_path):
    print("⚠️ Excel file not found at:", excel_path)
else:
    xl = pd.ExcelFile(excel_path)
    sheet_to_use = SHEET_NAME or xl.sheet_names[SHEET_INDEX-1]
    df_full = xl.parse(sheet_to_use)
    df = apply_row_limits(df_full)
    print(f"Evaluating sheet: {sheet_to_use} (rows: {len(df)})")

    print("\n>>>>> Original Prompt Response <<<<<<")
    # Core metrics
    if COL_WEED_GT in df and COL_WEED_PRED in df:
        wd_acc, wd_n = accuracy_from_series(df[COL_WEED_GT], df[COL_WEED_PRED],
                                            norm_weed_label, {"yes","no","n/a",""})
        print(f"Weed Detection Accuracy: {wd_acc:.4f} (n={wd_n})")

    if COL_GROWTH_GT in df and COL_GROWTH_PRED in df:
        cg_acc, cg_n = accuracy_from_series(df[COL_GROWTH_GT], df[COL_GROWTH_PRED],
                                            norm_growth, {"early","growing","full grown"})
        print(f"Crop Growth Accuracy: {cg_acc:.4f} (n={cg_n})")

    if COL_CROP_GT in df and COL_CROP_PRED in df:
        ct_acc, ct_n = accuracy_from_series(df[COL_CROP_GT], df[COL_CROP_PRED],
                                            norm_crop_type)
        print(f"Crop Type Accuracy: {ct_acc:.4f} (n={ct_n})")

    print("\n>>>>> EPP Response <<<<<<")
    # EPP metrics (by column letters)
    gt_e, gt_e_name = series_by_col_letter(df, "E")
    pr_o, pr_o_name = series_by_col_letter(df, "O")
    if gt_e is not None and pr_o is not None:
        epp_acc, epp_n = accuracy_from_series(gt_e, pr_o,
                                              norm_weed_label, {"yes","no","n/a",""})
        print(f"EPP Accuracy (GT {gt_e_name} vs Pred {pr_o_name}): {epp_acc:.4f} (n={epp_n})")

    gt_g, gt_g_name = series_by_col_letter(df, "G")
    pr_q, pr_q_name = series_by_col_letter(df, "Q")
    if gt_g is not None and pr_q is not None:
        epp_cg_acc, epp_cg_n = accuracy_from_series(gt_g, pr_q,
                                                    norm_growth, {"early","growing","full grown"})
        print(f"EPP Crop Growth Accuracy (GT {gt_g_name} vs Pred {pr_q_name}): {epp_cg_acc:.4f} (n={epp_cg_n})")

    gt_h, gt_h_name = series_by_col_letter(df, "H")
    pr_r, pr_r_name = series_by_col_letter(df, "R")
    if gt_h is not None and pr_r is not None:
        epp_ct_acc, epp_ct_n = accuracy_from_series(gt_h, pr_r, norm_crop_type)
        print(f"EPP Crop Type Accuracy (GT {gt_h_name} vs Pred {pr_r_name}): {epp_ct_acc:.4f} (n={epp_ct_n})")


Evaluating sheet: ChatGPT_4.1 (rows: 442)

>>>>> Original Prompt Response <<<<<<
Weed Detection Accuracy: 0.3100 (n=442)
Crop Growth Accuracy: 0.1927 (n=441)
Crop Type Accuracy: 0.5896 (n=441)

>>>>> EPP Response <<<<<<
EPP Accuracy (GT Weed Detection (GT) vs Pred V2_Weed Detection (Prediction)): 0.6923 (n=442)
EPP Crop Growth Accuracy (GT Crop Growth (GT) vs Pred V2_Crop Growth (Prediction)): 0.1541 (n=305)
EPP Crop Type Accuracy (GT Crop Type (GT) vs Pred V2_Crop Type (Prediction)): 0.6262 (n=305)


# Only GPT as Judge Cases - Evaluation of VLM Responses

## GPT Judge

In [ ]:


import os
import csv
import time
import base64
from io import BytesIO
from pathlib import Path
from typing import List, Tuple, Optional

from PIL import Image
from tqdm import tqdm

from openai import OpenAI  # pip install --upgrade openai

# =========================
# 🔐 CONFIG
# =========================
DOE = "" # Add API Key
MODEL = "gpt-4.1"              # vision-capable model (e.g., "gpt-4o")
IMAGE_DIR = Path(r"/Judge_Temp")
OUTPUT_CSV = Path(r"/Judge_Temp/WeedVLM_GPT_41_eval_outputs.csv")

# Behaviors
RESUME = True                 # skip rows already in CSV
MAX_RETRIES = 5               # API retry attempts
BASE_SLEEP = 2.0              # base seconds for exponential backoff: 2,4,8,16...
IMAGE_EXTS = {".jpg"}         # you said the images are .jpg (640x640)

# =========================
# Evaluation Prompt
# =========================
EVAL_PROMPT = """Now Rate the Response on the Following Evaluation Parameters:
Grounding: Does the explanation cite concrete, visible evidence (shape/colour/position) tied to specific regions?Score 5 if references are directly checkable in the image; 1 if evidence is generic or absent.
Specificity: How precise and unambiguous are the references (grid/row, counts, distances) versus vague words?Score 5 for exact, scannable pointers (“row 2, centre-left, ~0.3 of frame”); 1 for “somewhere left.”
Plausibility: Do the stated causes/effects make agronomic and visual sense given the scene? Score 5 when logic follows domain norms; 1 for leaps or contradictory reasoning. 
Non-Hallucination: Avoids mentioning objects/attributes not present or unverifiable from the image. Score 5 if every claim is image-supported; 1 if there are invented or misidentified elements 
Actionability: Would the text help a field operator quickly verify or act (where to look, what cue to confirm)?Score 5 if it gives clear next steps/checks; 1 if its not practically useful.
"""

# We will wrap the YOLO .txt content into the same user message so the model
# can consider detections while scoring.

# =========================
# Helpers
# =========================
def encode_image_to_base64_jpeg(image_path: Path) -> str:
    """
    Open image, convert to RGB, re-encode as JPEG, return base64 string.
    """
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        buf = BytesIO()
        img.save(buf, format="JPEG", quality=92)
        return base64.b64encode(buf.getvalue()).decode("utf-8")

def read_txt_content(txt_path: Path) -> str:
    """
    Read YOLO-format annotations from a .txt file and strip trailing whitespace.
    """
    with open(txt_path, "r", encoding="utf-8") as f:
        return f.read().strip()

def build_user_message_blocks(image_b64: str, yolo_text: str) -> list:
    """
    Construct the Chat Completions 'content' list that includes:
      - The evaluation rubric (EVAL_PROMPT)
      - The YOLO text content (embedded as plain text)
      - The image (data URL)
    """
    # We send the .txt *content* (not the file) in the prompt.
    yolo_note = (
        "Context: The following YOLO-format bounding boxes (class 0 = weed) "
        "are extracted from the paired .txt for this image:\n\n"
        f"{yolo_text}\n\n"
        "Use both the image and these annotations as context for your evaluation."
    )

    return [
        {"type": "text", "text": EVAL_PROMPT},
        {"type": "text", "text": yolo_note},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}},
    ]

def call_openai_vision(client: OpenAI, image_b64: str, yolo_text: str) -> str:
    """
    Single API call using Chat Completions, returning the first message text.
    """
    content_blocks = build_user_message_blocks(image_b64, yolo_text)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": content_blocks}],
        temperature=0.0,
        max_tokens=400,
    )
    return (resp.choices[0].message.content or "").strip()

def collect_images(dir_path: Path) -> List[Path]:
    return [p for p in sorted(dir_path.iterdir()) if p.suffix.lower() in IMAGE_EXTS and p.is_file()]

def find_txt_for_image(img_path: Path) -> Optional[Path]:
    """
    Return the .txt path with the same stem as the image (e.g., a.jpg -> a.txt).
    """
    txt_candidate = img_path.with_suffix(".txt")
    return txt_candidate if txt_candidate.exists() else None

def load_resume_set(csv_path: Path) -> set:
    """
    If RESUME is enabled and CSV exists, collect already-processed image names.
    """
    done = set()
    if RESUME and csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8", newline="") as f:
                reader = csv.reader(f)
                header = next(reader, None)
                if header:
                    name_idx = 0  # first column is image_name
                    for row in reader:
                        if row and len(row) > name_idx:
                            done.add(row[name_idx])
            print(f"Resuming: found {len(done)} rows in {csv_path}.")
        except Exception as e:
            print(f"Warning: resume failed to read CSV ({e}); starting fresh.")
    return done

# =========================
# Run Batch
# =========================
def main():
    client = OpenAI(api_key=DOE)

    IMAGE_DIR.mkdir(parents=True, exist_ok=True)
    images = collect_images(IMAGE_DIR)
    if not images:
        raise FileNotFoundError(f"No .jpg images found in: {IMAGE_DIR}")

    # Prepare CSV
    new_file = not OUTPUT_CSV.exists()
    with open(OUTPUT_CSV, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        if new_file:
            writer.writerow(["image_name", "gpt_response"])

        # Determine which images to skip if resuming
        already = load_resume_set(OUTPUT_CSV) if RESUME else set()

        for img_path in tqdm(images, desc="Processing images"):
            if img_path.name in already:
                continue

            txt_path = find_txt_for_image(img_path)
            if txt_path is None:
                # record an error row and continue
                writer.writerow([img_path.name, "ERROR: missing paired .txt file"])
                f.flush()
                continue

            # Read data
            try:
                yolo_text = read_txt_content(txt_path)
            except Exception as e:
                writer.writerow([img_path.name, f"ERROR: failed to read .txt: {e}"])
                f.flush()
                continue

            try:
                b64 = encode_image_to_base64_jpeg(img_path)
            except Exception as e:
                writer.writerow([img_path.name, f"ERROR: failed to encode image: {e}"])
                f.flush()
                continue

            # Query with retries
            out_text = None
            last_err = None
            for attempt in range(1, MAX_RETRIES + 1):
                try:
                    out_text = call_openai_vision(client, b64, yolo_text)
                    break
                except Exception as e:
                    last_err = str(e)
                    if attempt == MAX_RETRIES:
                        out_text = f"[ERROR after {MAX_RETRIES} retries] {last_err}"
                        break
                    # Exponential backoff
                    sleep_s = BASE_SLEEP * (2 ** (attempt - 1))
                    time.sleep(sleep_s)

            writer.writerow([img_path.name, out_text or ""])
            f.flush()

    print(f"Done. Wrote results to: {OUTPUT_CSV.resolve()}")

if __name__ == "__main__":
    main()


Resuming: found 0 rows in D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\Judge_Temp\WeedVLM_GPT_41_eval_outputs.csv.


Processing images: 100%|██████████| 20/20 [03:49<00:00, 11.45s/it]

Done. Wrote results to: D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\Judge_Temp\WeedVLM_GPT_41_eval_outputs.csv


### Working Code (Cross Checked)

In [ ]:
import os
import re
import csv
import time
import base64
from io import BytesIO
from pathlib import Path
from typing import List, Optional, Dict

import pandas as pd
from PIL import Image
from tqdm import tqdm
from openai import OpenAI

# =========================
# 🔐 CONFIG
# =========================
DOE = ""     # <-- put your real API key here
MODEL = "gpt-4.1"            # exactly ONE VLM used

IMAGE_DIR   = Path(r"/Judge_Temp")
OUTPUT_CSV  = Path(r"Judge_Temp/New_Predictions.csv")

EXCEL_PATH  = Path(r"/Weed-VLM Data Set/Complete_Simple_Data.xlsx")
SHEET_NAME  = "Gemini_Flash_2.5"  # pick the exact sheet name

RESUME      = True
MAX_RETRIES = 5
BASE_SLEEP  = 2.0
IMAGE_EXTS  = {".jpg"}

SHOW_PROMPT    = True
PROMPT_LOG_DIR = Path(OUTPUT_CSV.parent / "Prompt_Logs")

# =========================
# 📌 Evaluation Prompt
# =========================
EVAL_PROMPT = (
    "Now rate the *VLM response* for this image on the following (0–5 each):\n"
    "• Grounding – cites concrete, visible evidence tied to specific regions.\n"
    "• Specificity – precise, scannable references (rows/grid/offsets/counts).\n"
    "• Plausibility – agronomically sensible logic given the scene.\n"
    "• Non-Hallucination – no invented/irrelevant objects or attributes.\n"
    "• Actionability – clear cues/next steps for a field operator.\n\n"
    "IMPORTANT: The Input Imagee has red rectangles already drawn on the image are the ground truth weed locations. "
    "Use them as canonical when judging the VLM response."
    "No Symbols, Nothing Extra, Just Respond in this Manner: Grounding: <Score 0-5>, Specificity: <Score 0-5>, Plausibility: <Score 0-5>, Non-Hallucination: <Score 0-5>, Actionability: <Score 0-5>, "
)

# =========================
# 🧰 Helpers
# =========================
def encode_image_to_base64_jpeg(image_path: Path) -> str:
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        buf = BytesIO()
        img.save(buf, format="JPEG", quality=92)
        return base64.b64encode(buf.getvalue()).decode("utf-8")

def _norm(x: object) -> str:
    if pd.isna(x): return ""
    return str(x).strip()

def norm_key(s: str) -> str:
    """
    Robust comparable key for filenames:
    - lowercase, basename, drop image extension, remove non [a-z0-9]
    """
    s = str(s or "").strip().lower().replace("\\", "/")
    s = os.path.basename(s)
    if s.endswith((".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff")):
        s = Path(s).stem
    return re.sub(r"[^a-z0-9]+", "", s)

def is_yes(val: str) -> bool:
    v = (_norm(val)).lower()
    return v in {"yes", "y", "1", "true"}

def load_predictions_table(excel_path: Path, sheet_name: str) -> pd.DataFrame:
    """
    Try Excel first; if it fails, try CSV (in case it's a CSV with .xlsx extension).
    """
    try:
        xls = pd.ExcelFile(excel_path, engine="openpyxl")
        sheets = xls.sheet_names
        print("[INFO] Excel workbook sheets:", sheets)
        use_sheet = sheet_name if sheet_name in sheets else sheets[0]
        if use_sheet != sheet_name:
            print(f"[WARN] Sheet '{sheet_name}' not found. Using '{use_sheet}' instead.")
        df = pd.read_excel(xls, sheet_name=use_sheet, dtype=str)
        print(f"[INFO] Loaded predictions as Excel: {excel_path.name} | sheet='{use_sheet}' with {df.shape[0]} rows, {df.shape[1]} cols")
        return df
    except Exception:
        # CSV fallback
        df = pd.read_csv(excel_path, sep=None, engine="python", dtype=str)
        if df.shape[1] == 1:
            df2 = pd.read_csv(excel_path, sep=",", dtype=str)
            if df2.shape[1] > 1:
                df = df2
        print(f"[INFO] Loaded predictions as CSV: {excel_path.name} with {df.shape[0]} rows, {df.shape[1]} cols")
        return df

def read_spreadsheet_mapping(excel_path: Path, sheet_name: str) -> Dict[str, dict]:
    """
    Build mapping using explicit headers you provided:
      H: 'Weed Detection (Prediction)'
      I: 'Weed Location (Prediction)'
      L: 'Reasoning (Prediction)'
      M: 'V2_Weed Detection (Prediction)'
      N: 'V2_Weed Location (Prediction)'
      Q: 'V2_Reasoning (Prediction)'
    We keep both primary & V2 values in the row dict with clear keys.
    """
    df = load_predictions_table(excel_path, sheet_name).applymap(_norm)

    required_cols = [
        "Image Name",
        "Weed Detection (Prediction)",      # H
        "Weed Location (Prediction)",       # I
        "Reasoning (Prediction)",           # L
        "V2_Weed Detection (Prediction)",   # M
        "V2_Weed Location (Prediction)",    # N
        "V2_Reasoning (Prediction)",        # Q
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing expected columns in sheet: {missing}")

    # No data-row skipping (header already handled by pandas)
    df = df.copy()

    mapping: Dict[str, dict] = {}
    for _, row in df.iterrows():
        excel_img = _norm(row["Image Name"])  # e.g., 'Weed_Dataset (1)'
        if not excel_img or excel_img.lower() == "image name":  # guard
            continue

        key = norm_key(excel_img)  # 'weeddataset1'
        mapping[key] = {
            "excel_image_name": excel_img,
            "primary_detect":  _norm(row["Weed Detection (Prediction)"]),
            "primary_loc":     _norm(row["Weed Location (Prediction)"]),
            "primary_reason":  _norm(row["Reasoning (Prediction)"]),
            "v2_detect":       _norm(row["V2_Weed Detection (Prediction)"]),
            "v2_loc":          _norm(row["V2_Weed Location (Prediction)"]),
            "v2_reason":       _norm(row["V2_Reasoning (Prediction)"]),
        }

    print(f"[INFO] Loaded prediction mapping for ~{len(mapping)} images from '{excel_path.name}'.")
    print("[INFO] Detected columns:", list(df.columns))
    return mapping

def build_spreadsheet_context(sp_row: Optional[dict]) -> Optional[str]:
    """
    Branch selection:
      - If primary_detect is YES → use primary_loc + primary_reason
      - Else if v2_detect is YES → use v2_loc + v2_reason
      - Else → None
    """
    if not sp_row:
        return None

    if is_yes(sp_row.get("primary_detect", "")):
        loc = _norm(sp_row.get("primary_loc", ""))
        reas = _norm(sp_row.get("primary_reason", ""))
        return (
            "Spreadsheet Predictions (primary H columns):\n"
            f"- Weed Detection (H): Yes\n"
            f"- Weed Location (I): {loc}\n"
            f"- Reasoning (L): {reas}"
        )

    if is_yes(sp_row.get("v2_detect", "")):
        loc = _norm(sp_row.get("v2_loc", ""))
        reas = _norm(sp_row.get("v2_reason", ""))
        return (
            "Spreadsheet Predictions (fallback M columns):\n"
            f"- V2_Weed Detection (M): Yes\n"
            f"- V2_Weed Location (N): {loc}\n"
            f"- V2_Reasoning (Q): {reas}"
        )

    return None

def build_user_message_blocks(image_b64: str, spreadsheet_context: Optional[str]) -> list:
    blocks = [{"type": "text", "text": EVAL_PROMPT}]
    if spreadsheet_context:
        blocks.append({"type": "text", "text": spreadsheet_context})
    else:
        blocks.append({"type": "text", "text": "[Note] No spreadsheet prediction found for this image; score using GT boxes only."})
    blocks.append({"type": "text", "text": "Ground Truth: Red rectangles drawn on the image mark weed locations (canonical)."})
    blocks.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}})
    return blocks

def make_prompt_preview(image_path: Path, excel_image_name: Optional[str], spreadsheet_context: Optional[str]) -> str:
    parts = []
    parts.append("=== EVAL_PROMPT ===\n" + EVAL_PROMPT.strip())
    parts.append("\n=== Cross-check ===\n"
                 f"Disk image: {image_path.name}\n"
                 f"Excel 'Image Name': {excel_image_name or '[NO MATCH]'}")
    if spreadsheet_context:
        parts.append("\n=== Spreadsheet Context ===\n" + spreadsheet_context.strip())
    else:
        parts.append("\n=== Spreadsheet Context ===\n[NO MATCH FOUND]")
    parts.append("\n=== Ground Truth ===\nRed rectangles drawn on the image = weed locations.")
    parts.append(f"\n=== Image ===\n{image_path.name} (base64 bytes not shown)")
    return "\n".join(parts)

def call_openai_vision(client: OpenAI, image_b64: str, spreadsheet_context: Optional[str]) -> str:
    content_blocks = build_user_message_blocks(image_b64, spreadsheet_context)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": content_blocks}],
        temperature=0.0,
        max_tokens=600,
    )
    return (resp.choices[0].message.content or "").strip()

def collect_images(dir_path: Path) -> List[Path]:
    return [p for p in sorted(dir_path.iterdir()) if p.suffix.lower() in IMAGE_EXTS and p.is_file()]

def load_resume_set(csv_path: Path) -> set:
    done = set()
    if RESUME and csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8", newline="") as f:
                reader = csv.reader(f); next(reader, None)
                for row in reader:
                    if row: done.add(row[0])
            print(f"[INFO] Resuming: found {len(done)} rows in {csv_path}.")
        except Exception as e:
            print(f"[WARN] resume failed to read CSV ({e}); starting fresh.")
    return done

# =========================
# 🚀 Run Batch
# =========================
def main():
    client = OpenAI(api_key=DOE)

    # Load spreadsheet mapping (explicit headers; no row skip)
    sp_map = {}
    try:
        sp_map = read_spreadsheet_mapping(EXCEL_PATH, SHEET_NAME)
    except Exception as e:
        print(f"[WARN] failed to read predictions table ({e}). Proceeding without spreadsheet predictions.")

    IMAGE_DIR.mkdir(parents=True, exist_ok=True)
    images = collect_images(IMAGE_DIR)
    if not images:
        raise FileNotFoundError(f"No .jpg images found in: {IMAGE_DIR}")

    if SHOW_PROMPT:
        PROMPT_LOG_DIR.mkdir(parents=True, exist_ok=True)

    new_file = not OUTPUT_CSV.exists()
    with open(OUTPUT_CSV, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        if new_file:
            # includes excel_image_name for cross-check
            writer.writerow(["image_name", "excel_image_name", "gpt_response"])

        already = load_resume_set(OUTPUT_CSV) if RESUME else set()

        for img_path in tqdm(images, desc="Processing images"):
            if img_path.name in already:
                continue

            # Disk key and spreadsheet row
            key = norm_key(img_path.name)
            sp_row = sp_map.get(key)
            excel_img_name = sp_row.get("excel_image_name") if sp_row else None

            # Encode image
            try:
                b64 = encode_image_to_base64_jpeg(img_path)
            except Exception as e:
                writer.writerow([img_path.name, excel_img_name or "", f"ERROR: failed to encode image: {e}"])
                f.flush()
                continue

            # Build branch-specific context
            spreadsheet_context = build_spreadsheet_context(sp_row)

            # Optional: prompt preview (shows which branch was used and exact fields)
            if SHOW_PROMPT:
                preview = make_prompt_preview(img_path, excel_img_name, spreadsheet_context)
                print("\n" + "="*80)
                print(f"PROMPT PREVIEW for {img_path.name}")
                print("="*80)
                print(preview[:2000])
                (PROMPT_LOG_DIR / f"{img_path.stem}_prompt.txt").write_text(preview, encoding="utf-8")

            # ONE model call per image with retries
            out_text, last_err = None, None
            for attempt in range(1, MAX_RETRIES + 1):
                try:
                    out_text = call_openai_vision(client, b64, spreadsheet_context)
                    break
                except Exception as e:
                    last_err = str(e)
                    if attempt == MAX_RETRIES:
                        out_text = f"[ERROR after {MAX_RETRIES} retries] {last_err}"
                        break
                    time.sleep(BASE_SLEEP * (2 ** (attempt - 1)))

            writer.writerow([img_path.name, excel_img_name or "", out_text or ""])
            f.flush()

    print(f"Done. Wrote results to: {OUTPUT_CSV.resolve()}")

if __name__ == "__main__":
    main()


[INFO] Excel workbook sheets: ['Gemini_Flash_2.5', 'Gemini_Flash_Lite_2.5', 'LLaMA_4_Maverick', 'LLaMA_4_Scout', 'ChatGPT_4.1', 'ChatGPT_4o']
[INFO] Loaded predictions as Excel: Complete_Simple_Data.xlsx | sheet='Gemini_Flash_2.5' with 441 rows, 18 cols
[INFO] Loaded prediction mapping for ~441 images from 'Complete_Simple_Data.xlsx'.
[INFO] Detected columns: ['#', 'Image Name', 'Weed Detection (GT)', 'Weed Location (GT)', 'Crop Growth (GT)', 'Crop Type (GT)', 'Gemini_Response', 'Weed Detection (Prediction)', 'Weed Location (Prediction)', 'Crop Growth (Prediction)', 'Crop Type (Prediction)', 'Reasoning (Prediction)', 'V2_Weed Detection (Prediction)', 'V2_Weed Location (Prediction)', 'V2_Crop Growth (Prediction)', 'V2_Crop Type (Prediction)', 'V2_Reasoning (Prediction)', 'Weed Detection Penalty (Prompt Entry)']
[INFO] Resuming: found 0 rows in D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\Judge_Temp\New_Predictions.csv.


Processing images:   0%|          | 0/5 [00:00<?, ?it/s]


PROMPT PREVIEW for Weed_Dataset (1).jpg
=== EVAL_PROMPT ===
Now rate the *VLM response* for this image on the following (0–5 each):
• Grounding – cites concrete, visible evidence tied to specific regions.
• Specificity – precise, scannable references (rows/grid/offsets/counts).
• Plausibility – agronomically sensible logic given the scene.
• Non-Hallucination – no invented/irrelevant objects or attributes.
• Actionability – clear cues/next steps for a field operator.

IMPORTANT: The Input Imagee has red rectangles already drawn on the image are the ground truth weed locations. Use them as canonical when judging the VLM response.No Symbols, Nothing Extra, Just Respond in this Manner: Grounding: <Score 0-5>, Specificity: <Score 0-5>, Plausibility: <Score 0-5>, Non-Hallucination: <Score 0-5>, Actionability: <Score 0-5>,

=== Cross-check ===
Disk image: Weed_Dataset (1).jpg
Excel 'Image Name': Weed_Dataset (1)

=== Spreadsheet Context ===
Spreadsheet Predictions (fallback M columns):
- V2

Processing images:  20%|██        | 1/5 [00:03<00:13,  3.32s/it]


PROMPT PREVIEW for Weed_Dataset (2).jpg
=== EVAL_PROMPT ===
Now rate the *VLM response* for this image on the following (0–5 each):
• Grounding – cites concrete, visible evidence tied to specific regions.
• Specificity – precise, scannable references (rows/grid/offsets/counts).
• Plausibility – agronomically sensible logic given the scene.
• Non-Hallucination – no invented/irrelevant objects or attributes.
• Actionability – clear cues/next steps for a field operator.

IMPORTANT: The Input Imagee has red rectangles already drawn on the image are the ground truth weed locations. Use them as canonical when judging the VLM response.No Symbols, Nothing Extra, Just Respond in this Manner: Grounding: <Score 0-5>, Specificity: <Score 0-5>, Plausibility: <Score 0-5>, Non-Hallucination: <Score 0-5>, Actionability: <Score 0-5>,

=== Cross-check ===
Disk image: Weed_Dataset (2).jpg
Excel 'Image Name': Weed_Dataset (2)

=== Spreadsheet Context ===
Spreadsheet Predictions (primary H columns):
- Wee

...


PROMPT PREVIEW for Weed_Dataset (4).jpg
=== EVAL_PROMPT ===
Now rate the *VLM response* for this image on the following (0–5 each):
• Grounding – cites concrete, visible evidence tied to specific regions.
• Specificity – precise, scannable references (rows/grid/offsets/counts).
• Plausibility – agronomically sensible logic given the scene.
• Non-Hallucination – no invented/irrelevant objects or attributes.
• Actionability – clear cues/next steps for a field operator.

IMPORTANT: The Input Imagee has red rectangles already drawn on the image are the ground truth weed locations. Use them as canonical when judging the VLM response.No Symbols, Nothing Extra, Just Respond in this Manner: Grounding: <Score 0-5>, Specificity: <Score 0-5>, Plausibility: <Score 0-5>, Non-Hallucination: <Score 0-5>, Actionability: <Score 0-5>,

=== Cross-check ===
Disk image: Weed_Dataset (4).jpg
Excel 'Image Name': Weed_Dataset (4)

=== Spreadsheet Context ===
Spreadsheet Predictions (primary H columns):
- Wee

Processing images:  80%|████████  | 4/5 [00:15<00:03,  3.82s/it]


PROMPT PREVIEW for Weed_Dataset (5).jpg
=== EVAL_PROMPT ===
Now rate the *VLM response* for this image on the following (0–5 each):
• Grounding – cites concrete, visible evidence tied to specific regions.
• Specificity – precise, scannable references (rows/grid/offsets/counts).
• Plausibility – agronomically sensible logic given the scene.
• Non-Hallucination – no invented/irrelevant objects or attributes.
• Actionability – clear cues/next steps for a field operator.

IMPORTANT: The Input Imagee has red rectangles already drawn on the image are the ground truth weed locations. Use them as canonical when judging the VLM response.No Symbols, Nothing Extra, Just Respond in this Manner: Grounding: <Score 0-5>, Specificity: <Score 0-5>, Plausibility: <Score 0-5>, Non-Hallucination: <Score 0-5>, Actionability: <Score 0-5>,

=== Cross-check ===
Disk image: Weed_Dataset (5).jpg
Excel 'Image Name': Weed_Dataset (5)

=== Spreadsheet Context ===
Spreadsheet Predictions (primary H columns):
- Wee

Processing images: 100%|██████████| 5/5 [00:18<00:00,  3.76s/it]

Done. Wrote results to: D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\Judge_Temp\New_Predictions.csv


## Prompt Simplified Code

In [ ]:
import os
import re
import csv
import time
import base64
from io import BytesIO
from pathlib import Path
from typing import List, Optional, Dict

import pandas as pd
from PIL import Image
from tqdm import tqdm
from openai import OpenAI

# =========================
# 🔐 CONFIG
# =========================

DOE = ""     # <-- put your real API key here
MODEL = "gpt-4.1"            # exactly ONE VLM used

IMAGE_DIR   = Path(r"/Judge_Temp")
OUTPUT_CSV  = Path(r"Judge_Temp/New_Predictions.csv")

EXCEL_PATH  = Path(r"/Weed-VLM Data Set/Complete_Simple_Data.xlsx")
SHEET_NAME  = "Gemini_Flash_2.5"  # pick the exact sheet name

RESUME      = True
MAX_RETRIES = 5
BASE_SLEEP  = 2.0
IMAGE_EXTS  = {".jpg"}

# Disable all console prints to keep tqdm clean
VERBOSE = False

# Keep logs of sent prompts? (saved to files only; no printing)
SHOW_PROMPT    = False
PROMPT_LOG_DIR = Path(OUTPUT_CSV.parent / "Prompt_Logs")

# =========================
# 📌 Evaluation Prompt
# =========================
EVAL_PROMPT = (
    "Now rate the *VLM response* for this image on the following (0–5 each):\n"
    "• Grounding – cites concrete, visible evidence tied to specific regions.\n"
    "• Specificity – precise, scannable references (rows/grid/offsets/counts).\n"
    "• Plausibility – agronomically sensible logic given the scene.\n"
    "• Non-Hallucination – no invented/irrelevant objects or attributes.\n"
    "• Actionability – clear cues/next steps for a field operator.\n\n"
    "IMPORTANT: The input image has red rectangles already drawn as the ground-truth weed locations. "
    "Use them as canonical when judging the VLM response. "
    "No symbols, nothing extra, No Need to Add Rationale, Never anything Additional, In a Single Line, Always Respond exactly as: Grounding: <0-5>, Specificity: <0-5>, Plausibility: <0-5>, Non-Hallucination: <0-5>, Actionability: <0-5>"
)

# =========================
# 🧰 Helpers
# =========================
def encode_image_to_base64_jpeg(image_path: Path) -> str:
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        buf = BytesIO()
        img.save(buf, format="JPEG", quality=92)
        return base64.b64encode(buf.getvalue()).decode("utf-8")

def _norm(x: object) -> str:
    if pd.isna(x): return ""
    return str(x).strip()

def norm_key(s: str) -> str:
    """
    Robust comparable key for filenames:
    - lowercase, basename, drop image extension, remove non [a-z0-9]
    """
    s = str(s or "").strip().lower().replace("\\", "/")
    s = os.path.basename(s)
    if s.endswith((".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff")):
        s = Path(s).stem
    return re.sub(r"[^a-z0-9]+", "", s)

def is_yes(val: str) -> bool:
    v = (_norm(val)).lower()
    return v in {"yes", "y", "1", "true"}

def load_predictions_table(excel_path: Path, sheet_name: str) -> pd.DataFrame:
    """
    Try Excel first; if it fails, try CSV (in case it's a CSV with .xlsx extension).
    """
    try:
        xls = pd.ExcelFile(excel_path, engine="openpyxl")
        use_sheet = sheet_name if sheet_name in xls.sheet_names else xls.sheet_names[0]
        df = pd.read_excel(xls, sheet_name=use_sheet, dtype=str)
        return df
    except Exception:
        df = pd.read_csv(excel_path, sep=None, engine="python", dtype=str)
        if df.shape[1] == 1:
            df2 = pd.read_csv(excel_path, sep=",", dtype=str)
            if df2.shape[1] > 1:
                df = df2
        return df

def read_spreadsheet_mapping(excel_path: Path, sheet_name: str) -> Dict[str, dict]:
    """
    Build mapping using explicit headers:
      H: 'Weed Detection (Prediction)'
      I: 'Weed Location (Prediction)'
      L: 'Reasoning (Prediction)'
      M: 'V2_Weed Detection (Prediction)'
      N: 'V2_Weed Location (Prediction)'
      Q: 'V2_Reasoning (Prediction)'
    """
    df = load_predictions_table(excel_path, sheet_name).applymap(_norm)

    required_cols = [
        "Image Name",
        "Weed Detection (Prediction)",      # H
        "Weed Location (Prediction)",       # I
        "Reasoning (Prediction)",           # L
        "V2_Weed Detection (Prediction)",   # M
        "V2_Weed Location (Prediction)",    # N
        "V2_Reasoning (Prediction)",        # Q
    ]
    for c in required_cols:
        if c not in df.columns:
            raise KeyError(f"Missing expected column: {c}")

    mapping: Dict[str, dict] = {}
    for _, row in df.iterrows():
        excel_img = _norm(row["Image Name"])  # e.g., 'Weed_Dataset (1)'
        if not excel_img or excel_img.lower() == "image name":
            continue

        key = norm_key(excel_img)  # 'weeddataset1'
        mapping[key] = {
            "excel_image_name": excel_img,
            "primary_detect":  _norm(row["Weed Detection (Prediction)"]),
            "primary_loc":     _norm(row["Weed Location (Prediction)"]),
            "primary_reason":  _norm(row["Reasoning (Prediction)"]),
            "v2_detect":       _norm(row["V2_Weed Detection (Prediction)"]),
            "v2_loc":          _norm(row["V2_Weed Location (Prediction)"]),
            "v2_reason":       _norm(row["V2_Reasoning (Prediction)"]),
        }
    return mapping

def build_spreadsheet_context(sp_row: Optional[dict]) -> Optional[str]:
    """
    Branch selection:
      - If primary_detect is YES → use primary_loc + primary_reason
      - Else if v2_detect is YES → use v2_loc + v2_reason
      - Else → None
    """
    if not sp_row:
        return None

    if is_yes(sp_row.get("primary_detect", "")):
        loc = _norm(sp_row.get("primary_loc", ""))
        reas = _norm(sp_row.get("primary_reason", ""))
        return (
            "Spreadsheet Predictions (primary H columns):\n"
            f"- Weed Detection (H): Yes\n"
            f"- Weed Location (I): {loc}\n"
            f"- Reasoning (L): {reas}"
        )

    if is_yes(sp_row.get("v2_detect", "")):
        loc = _norm(sp_row.get("v2_loc", ""))
        reas = _norm(sp_row.get("v2_reason", ""))
        return (
            "Spreadsheet Predictions (fallback M columns):\n"
            f"- V2_Weed Detection (M): Yes\n"
            f"- V2_Weed Location (N): {loc}\n"
            f"- V2_Reasoning (Q): {reas}"
        )

    return None

def build_user_message_blocks(image_b64: str, spreadsheet_context: Optional[str]) -> list:
    blocks = [{"type": "text", "text": EVAL_PROMPT}]
    if spreadsheet_context:
        blocks.append({"type": "text", "text": spreadsheet_context})
    else:
        blocks.append({"type": "text", "text": "[Note] No spreadsheet prediction found for this image; score using GT boxes only."})
    blocks.append({"type": "text", "text": "Ground Truth: Red rectangles drawn on the image mark weed locations (canonical)."})
    blocks.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}})
    return blocks

def call_openai_vision(client: OpenAI, image_b64: str, spreadsheet_context: Optional[str]) -> str:
    content_blocks = build_user_message_blocks(image_b64, spreadsheet_context)
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": content_blocks}],
        temperature=0.0,
        max_tokens=600,
    )
    return (resp.choices[0].message.content or "").strip()

def collect_images(dir_path: Path) -> List[Path]:
    return [p for p in sorted(dir_path.iterdir()) if p.suffix.lower() in IMAGE_EXTS and p.is_file()]

def load_resume_set(csv_path: Path) -> set:
    done = set()
    if RESUME and csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8", newline="") as f:
                reader = csv.reader(f); next(reader, None)
                for row in reader:
                    if row: done.add(row[0])
        except Exception:
            pass
    return done

# =========================
# 🚀 Run Batch
# =========================
def main():
    client = OpenAI(api_key=DOE)

    # Load spreadsheet mapping (explicit headers; no row skip)
    try:
        sp_map = read_spreadsheet_mapping(EXCEL_PATH, SHEET_NAME)
    except Exception:
        sp_map = {}

    IMAGE_DIR.mkdir(parents=True, exist_ok=True)
    images = collect_images(IMAGE_DIR)
    if not images:
        raise FileNotFoundError(f"No .jpg images found in: {IMAGE_DIR}")

    if SHOW_PROMPT:
        PROMPT_LOG_DIR.mkdir(parents=True, exist_ok=True)

    new_file = not OUTPUT_CSV.exists()
    with open(OUTPUT_CSV, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        if new_file:
            # ✅ Added 4th column: spreadsheet_context
            writer.writerow(["image_name", "excel_image_name", "gpt_response", "spreadsheet_context"])

        already = load_resume_set(OUTPUT_CSV) if RESUME else set()

        for img_path in tqdm(images, desc="Processing images"):
            if img_path.name in already:
                continue

            key = norm_key(img_path.name)
            sp_row = sp_map.get(key)
            excel_img_name = sp_row.get("excel_image_name") if sp_row else ""

            try:
                b64 = encode_image_to_base64_jpeg(img_path)
            except Exception as e:
                writer.writerow([img_path.name, excel_img_name, f"ERROR: failed to encode image: {e}", ""])
                f.flush()
                continue

            spreadsheet_context = build_spreadsheet_context(sp_row)

            # (Optional) Save prompt preview to file only (no prints)
            if SHOW_PROMPT:
                preview = (
                    "=== EVAL_PROMPT ===\n" + EVAL_PROMPT.strip() +
                    "\n\n=== Cross-check ===\n"
                    f"Disk image: {img_path.name}\n"
                    f"Excel 'Image Name': {excel_img_name or '[NO MATCH]'}\n" +
                    ("\n=== Spreadsheet Context ===\n" + spreadsheet_context if spreadsheet_context else
                     "\n=== Spreadsheet Context ===\n[NO MATCH FOUND]") +
                    "\n\n=== Ground Truth ===\nRed rectangles drawn on the image = weed locations." +
                    f"\n\n=== Image ===\n{img_path.name} (base64 bytes not shown)"
                )
                (PROMPT_LOG_DIR / f"{img_path.stem}_prompt.txt").write_text(preview, encoding="utf-8")

            # ONE model call per image with retries
            out_text = None
            last_err = None
            for attempt in range(1, MAX_RETRIES + 1):
                try:
                    out_text = call_openai_vision(client, b64, spreadsheet_context)
                    break
                except Exception as e:
                    last_err = str(e)
                    if attempt == MAX_RETRIES:
                        out_text = f"[ERROR after {MAX_RETRIES} retries] {last_err}"
                        break
                    time.sleep(BASE_SLEEP * (2 ** (attempt - 1)))

            # ✅ Write the spreadsheet_context into the CSV as the 4th column
            writer.writerow([img_path.name, excel_img_name, out_text or "", spreadsheet_context or ""])
            f.flush()

if __name__ == "__main__":
    main()


Processing images: 100%|██████████| 441/441 [22:10<00:00,  3.02s/it]
